In [98]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD


In [99]:
# Loading datasets
ratings = pd.read_csv("../backend/data/ratings.csv")  # userId, movieId, rating, timestamp
movies = pd.read_csv("../backend/data/movies.csv")  # movieId, title

# Create a dictionary mapping movieId to title
movie_id_to_title = dict(zip(movies['movieId'], movies['title']))


## Step 1: Preprocessing Block

In [100]:
def preprocess_ratings(ratings, movies):
    """
    Preprocess the ratings data to create user-item matrix and ID mappings.
    
    :param ratings: DataFrame with user-item ratings
    :param movies: DataFrame with movie data
    
    :return: A tuple containing the user-item matrix and ID mappings
    """
    # Movie ID to index map
    movie_id_to_index = {movie_id: index for index, movie_id in enumerate(movies['movieId'])}
    index_to_movie_id = {index: movie_id for movie_id, index in movie_id_to_index.items()}
    
    # User ID to index map
    user_id_to_index = {user_id: index for index, user_id in enumerate(ratings['userId'].unique())}
    index_to_user_id = {index: user_id for user_id, index in user_id_to_index.items()}
    
    # Map movie and user IDs to indices
    ratings['movieIndex'] = ratings['movieId'].map(movie_id_to_index)
    ratings['userIndex'] = ratings['userId'].map(user_id_to_index)
    
    # Create the user-item matrix
    user_movie_matrix = ratings.pivot(index='userIndex', columns='movieIndex', values='rating').fillna(0)
    
    return user_movie_matrix, movie_id_to_index, index_to_movie_id, user_id_to_index, index_to_user_id


In [101]:
user_movie_matrix, movie_id_to_index, index_to_movie_id, user_id_to_index, index_to_user_id = preprocess_ratings(ratings=ratings,movies=movies)

## Step 2: Recommendation Functions 

#### 1. SVD Recommendation

In [102]:
def svd_recommendation(user_movie_matrix,  n_components=100):
    """
    Apply SVD for matrix factorization using preprocessed data.
    :param user_movie_matrix: The preprocessed user-item matrix
    :param n_components: Number of latent features to learn
    
    :return: A matrix of predictions (user-item)
    """
    # Apply SVD
    svd = TruncatedSVD(n_components=n_components,random_state=10)
    latent_matrix = svd.fit_transform(user_movie_matrix)
    
    # Get predictions
    prediction_matrix = np.dot(latent_matrix, svd.components_)
    
    return prediction_matrix


In [103]:
user_movie_matrix

movieIndex,0,1,2,3,4,5,6,7,8,9,...,9732,9733,9734,9735,9736,9737,9738,9739,9740,9741
userIndex,,,,,,,,,,,,,,,,,,,,,
0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
606,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
svd_prediction_matrix=svd_recommendation(user_movie_matrix,n_components=100)
pd.DataFrame(svd_prediction_matrix)

,0,1,2,3,4,5,6,7,8,9,...,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723
0,3.777933,0.579176,3.239313,0.167166,-0.216358,3.710185,-0.246727,0.124543,0.225833,-1.051540,...,0.058879,0.050468,0.067290,0.067290,0.058879,0.067290,0.058879,0.058879,0.058879,-0.241209
1,0.404520,0.104869,0.042348,-0.017279,0.140782,-0.117847,0.037218,0.042399,0.054366,-0.014899,...,0.027851,0.023872,0.031829,0.031829,0.027851,0.031829,0.027851,0.027851,0.027851,0.044882
2,0.067805,-0.050554,0.083630,0.002748,0.127446,0.200956,0.067487,0.018059,0.023367,-0.026370,...,-0.005838,-0.005004,-0.006672,-0.006672,-0.005838,-0.006672,-0.005838,-0.005838,-0.005838,-0.000807
3,1.744598,0.115580,0.263414,0.123071,0.488841,0.458420,0.562640,0.079553,-0.145587,-0.224163,...,0.049793,0.042680,0.056906,0.056906,0.049793,0.056906,0.049793,0.049793,0.049793,-0.002157
4,1.914832,0.858951,-0.178218,0.043944,0.115782,0.615337,0.211471,0.046749,-0.170759,0.654323,...,-0.013937,-0.011946,-0.015928,-0.015928,-0.013937,-0.015928,-0.013937,-0.013937,-0.013937,0.011197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2.402352,-0.019061,0.065262,0.018725,-0.070926,-0.061577,2.554138,0.002555,0.057150,-0.254558,...,-0.025326,-0.021708,-0.028944,-0.028944,-0.025326,-0.028944,-0.025326,-0.025326,-0.025326,-0.044428
606,3.616935,-0.225266,0.521178,0.091948,0.041682,0.522931,-0.575799,0.049259,0.437759,-0.002607,...,0.037183,0.031871,0.042495,0.042495,0.037183,0.042495,0.037183,0.037183,0.037183,-0.110314
607,0.801622,2.443100,1.670834,0.081390,-0.407588,-0.013157,-0.116424,-0.066236,0.215761,2.922354,...,-0.034118,-0.029244,-0.038992,-0.038992,-0.034118,-0.038992,-0.034118,-0.034118,-0.034118,0.123717
608,1.510505,0.439842,-0.069015,-0.091086,0.124576,0.217002,-0.007176,-0.029331,0.084732,1.332157,...,0.000014,0.000012,0.000016,0.000016,0.000014,0.000016,0.000014,0.000014,0.000014,-0.010114


#### 2. User-User Collaborative Filtering

In [105]:
def user_user_collaborative_filtering( user_movie_matrix,  k=5):
    """
    Recommend movies based on user-user similarity (k-nearest neighbors) using preprocessed data.
    
    :param user_movie_matrix: The preprocessed user-item matrix
    :param k: The number of nearest neighbors to consider
    
    :return: A dictionary of userId -> recommended movie IDs
    """
    # Compute the user-user similarity using k-NN
    knn = NearestNeighbors(metric='cosine', algorithm='brute')
    knn.fit(user_movie_matrix)
    
    recommendations = {}
    
    for user_id in user_movie_matrix.index:
        distances, indices = knn.kneighbors(user_movie_matrix.loc[user_id].values.reshape(1, -1), n_neighbors=k)
        
        # Get movies rated highly by similar users
        similar_users = user_movie_matrix.iloc[indices[0]]
        recommended_movie_indices = similar_users.mean(axis=0).sort_values(ascending=False).index[:10]
        
        # Convert movie indices back to movie IDs
        recommended_movie_ids = [index_to_movie_id[idx] for idx in recommended_movie_indices]
        recommendations[index_to_user_id[user_id]] = recommended_movie_ids
    
    return recommendations


In [106]:
user_user_recommendations = user_user_collaborative_filtering(user_movie_matrix)

In [107]:
user_user_recommendations

{np.int64(1): [1198, 2571, 1222, 50, 2028, 1197, 608, 1275, 1208, 1240],
 np.int64(2): [79132,
  58559,
  109487,
  106782,
  318,
  7153,
  68157,
  4993,
  3578,
  74458],
 np.int64(3): [1214, 1371, 3703, 1587, 1200, 593, 1198, 849, 1291, 2288],
 np.int64(4): [912, 908, 2997, 904, 608, 1265, 1179, 2858, 1219, 1225],
 np.int64(5): [527, 296, 457, 590, 34, 318, 380, 150, 300, 356],
 np.int64(6): [356, 110, 588, 480, 457, 277, 168, 377, 595, 368],
 np.int64(7): [356, 260, 1196, 1210, 4306, 480, 8961, 296, 780, 2571],
 np.int64(8): [457, 150, 110, 590, 356, 364, 480, 380, 253, 318],
 np.int64(9): [5952, 1198, 1270, 4993, 2571, 1196, 1210, 3996, 3578, 260],
 np.int64(10): [356, 8533, 4447, 68954, 597, 81845, 4306, 106696, 5066, 6377],
 np.int64(11): [457, 318, 110, 150, 349, 589, 153, 329, 292, 780],
 np.int64(12): [5620, 2694, 2572, 6942, 39, 1721, 261, 356, 1380, 2005],
 np.int64(13): [3578, 2571, 3793, 3717, 3996, 1210, 1198, 3623, 4011, 4993],
 np.int64(14): [110, 296, 318, 593, 47, 3

#### 3. Item-Item Collaborative Filtering

In [108]:
def item_item_collaborative_filtering(user_movie_matrix, k=5):
    """
    Recommend movies based on item-item similarity (k-nearest neighbors) using preprocessed data.
    
    :param user_movie_matrix: The preprocessed user-item matrix
    :param k: The number of nearest neighbors to consider
    
    :return: A dictionary of movieId -> recommended movie IDs
    """
    # Compute the item-item similarity using k-NN
    knn = NearestNeighbors(metric='cosine', algorithm='brute')
    knn.fit(user_movie_matrix.T)  # Transpose to make movies as rows
    
    recommendations = {}
    
    for movie_id in user_movie_matrix.columns:
        distances, indices = knn.kneighbors(user_movie_matrix.T.loc[movie_id].values.reshape(1, -1), n_neighbors=k)
        
        # Get similar movies based on the user ratings
        similar_movie_indices = user_movie_matrix.columns[indices[0]]
        
        # Convert movie indices back to movie IDs
        similar_movie_ids = [index_to_movie_id[idx] for idx in similar_movie_indices]
        recommendations[index_to_movie_id[movie_id]] = similar_movie_ids
    
    return recommendations


In [109]:
item_tem_recommendations=item_item_collaborative_filtering(user_movie_matrix)

In [110]:
item_tem_recommendations 

{1: [1, 3114, 480, 780, 260],
 2: [2, 364, 500, 367, 480],
 3: [3, 3450, 762, 788, 736],
 4: [4, 113, 987, 55, 722],
 5: [5, 7, 79, 762, 62],
 6: [6, 733, 32, 293, 16],
 7: [7, 708, 5, 62, 141],
 8: [8, 271, 174, 502, 217],
 9: [9, 634, 786, 74, 667],
 10: [10, 165, 380, 349, 377],
 11: [11, 597, 440, 539, 587],
 12: [12, 39400, 6263, 6379, 27311],
 13: [13, 1489, 1919, 1547, 6358],
 14: [14, 52, 36, 25, 628],
 15: [15, 93, 340, 415, 795],
 16: [16, 1213, 6, 4262, 555],
 17: [17, 36, 838, 141, 224],
 18: [18, 5298, 2691, 3992, 5628],
 19: [19, 344, 367, 231, 2],
 20: [20, 27478, 32213, 26870, 95583],
 21: [21, 440, 457, 357, 539],
 22: [22, 257, 218, 350, 186],
 23: [23, 174, 325, 259, 276],
 24: [24, 577, 2846, 4519, 3111],
 25: [25, 36, 14, 32, 6],
 26: [26, 41, 100, 273, 314],
 27: [27, 191, 1006, 71619, 91079],
 28: [28, 2177, 6183, 3475, 1545],
 29: [29, 1175, 1274, 1199, 1748],
 30: [30, 2024, 190, 3158, 3129],
 31: [31, 371, 191, 277, 515],
 32: [32, 296, 589, 780, 47],
 34: [34

## Revised Scenarios
#### 1. New User (No History)

In [111]:
def new_user_recommendation_with_diversity(
    ratings, 
    item_item_recommendations, 
    watched_movie_id=None, 
    top_n=10, 
    diversity_factor=0.3
):
    """
    Recommends movies for a new user with a focus on diversity.
    - If no movie is watched, recommend a mix of popular and random movies.
    - If a movie is watched, recommend a mix of similar and diverse movies.
    
    :param ratings: DataFrame with columns ['userId', 'movieId', 'rating']
    :param item_item_recommendations: Precomputed dictionary of movieId -> recommended movie IDs
    :param watched_movie_id: ID of the movie watched by the new user
    :param top_n: Number of movies to recommend
    :param diversity_factor: Proportion of recommendations to include as random or diverse
    
    :return: List of recommended movie IDs
    """
    if watched_movie_id is None:
        # Popular movies
        popular_movies = (
            ratings.groupby('movieId')['rating']
            .mean()
            .sort_values(ascending=False)
            .head(top_n)
            .index.tolist()
        )
        
        # Add diversity by mixing in some random movies
        all_movies = ratings['movieId'].unique()
        num_diverse = int(top_n * diversity_factor)
        random_movies = list(np.random.choice(all_movies, size=num_diverse, replace=False))
        
        return popular_movies[:top_n - num_diverse] + random_movies
    
    else:
        # Similar movies based on item-item recommendations
        if watched_movie_id in item_item_recommendations:
            similar_movies = item_item_recommendations[watched_movie_id][:top_n]
            print("get similar movie... ")
        else:
            print("no movie id!")
            similar_movies = []
        
        # Add diversity by mixing in random movies
        all_movies = ratings['movieId'].unique()
        num_diverse = int(top_n * diversity_factor)
        random_movies = list(np.random.choice(all_movies, size=num_diverse, replace=False))
        
        return similar_movies[:top_n - num_diverse] + random_movies


In [112]:
new_user_recommendation_with_diversity(ratings,item_tem_recommendations,top_n=10)

[187717,
 6983,
 5328,
 95843,
 3941,
 3940,
 3939,
 np.int64(5483),
 np.int64(3937),
 np.int64(46)]

## 2. Returning User (History Exists)
#### Home Page Recommendations

In [113]:
import random

def home_page_recommendations_with_diversity(
    user_id, 
    ratings, 
    user_movie_matrix, 
    item_item_recommendations, 
    user_user_recommendations, 
    svd_prediction_matrix, 
    weights=(0.4, 0.3, 0.3), 
    top_n=50, 
    global_top_movies=5  # Number of global top rated movies to include
):
    """
    Generates home page recommendations for a returning user based on their history, 
    with randomization, diversity, and sorting by predicted and average ratings.
    
    :param user_id: ID of the user for whom to generate recommendations
    :param ratings: DataFrame with columns ['userId', 'movieId', 'rating']
    :param user_movie_matrix: User-item matrix
    :param item_item_recommendations: Precomputed item-item recommendations
    :param user_user_recommendations: Precomputed user-user recommendations
    :param svd_prediction_matrix: SVD-based prediction matrix
    :param weights: Weights for blending the recommendation methods (item-item, user-user, SVD)
    :param top_n: Number of movies to recommend
    :param diversity_factor: Factor to control diversity in the recommendations (between 0 and 1)
    :param global_top_movies: Number of global top rated movies to include
    :return: List of recommended movie IDs
    """
    
    # Get the user's ratings history
    user_ratings = ratings[ratings['userId'] == user_id]
    rated_movies = user_ratings['movieId'].tolist()
    
    # ----- 1. Item-Item Collaborative Filtering -----
    top_rated_movies = user_ratings.sort_values(by='rating', ascending=False)['movieId'].head(5).tolist()
    item_item_movies = []
    for movie_id in top_rated_movies:
        if movie_id in item_item_recommendations:
            item_item_movies.extend(item_item_recommendations[movie_id])
    
    item_item_movies = [m for m in item_item_movies if m not in rated_movies]
    
    # ----- 2. User-User Collaborative Filtering -----
    user_user_movies = []
    if user_id in user_user_recommendations:
        similar_user_top_movies = user_user_recommendations[user_id]
        user_user_movies.extend([m for m in similar_user_top_movies if m not in rated_movies])
    
    # ----- 3. Predicted Ratings (SVD) -----
    user_index = user_id_to_index[user_id]
    if user_index in user_movie_matrix.index:
        svd_predictions = svd_prediction_matrix[user_index]
        svd_movie_ids = [index_to_movie_id[idx] for idx in user_movie_matrix.columns]
        svd_recommendations = sorted(
            zip(svd_movie_ids, svd_predictions), 
            key=lambda x: x[1], 
            reverse=True
        )
        svd_movies = [movie_id for movie_id, pred in svd_recommendations if movie_id not in rated_movies]
    else:
        svd_movies = []
    
    # ----- Global Top Rated Movies -----
    popular_movies = ratings.groupby('movieId')['rating'].mean().sort_values(ascending=False).head(global_top_movies).index.tolist()
    popular_movies = [m for m in popular_movies if m not in rated_movies]
    
    # ----- Blending Recommendations -----
    item_item_count = int(weights[0] * top_n)
    user_user_count = int(weights[1] * top_n)
    svd_count = top_n - item_item_count - user_user_count
    
    # Combine recommendations with weights
    combined_recommendations = (
        item_item_movies[:item_item_count] + 
        user_user_movies[:user_user_count] + 
        svd_movies[:svd_count]
    )
    
    # Add Global Top Rated Movies
    combined_recommendations = list(set(combined_recommendations) | set(popular_movies))
    
    # Introduce diversity by randomly shuffling the combined recommendations
    random.shuffle(combined_recommendations)
    
    # Get the top N recommended movies, considering the total size
    final_recommendations = combined_recommendations[:top_n]
    
    # ----- Get the predicted ratings for each movie in the final recommendations
    
    predicted_ratings = []
    for movie_id in final_recommendations:
        # Check if we have a predicted rating for the movie
        if movie_id in svd_movies:
            movie_index = movie_id_to_index[movie_id]
            predicted_rating = svd_prediction_matrix[user_index][movie_index]
        else:
            predicted_rating = 0  # Default to 0 if no prediction is available
        
        predicted_ratings.append((movie_id, predicted_rating))
    
    # Sort the recommendations by the predicted ratings
    final_recommendations = sorted(predicted_ratings, key=lambda x: x[1], reverse=True)
    
    # Return only the movie IDs sorted by their predicted ratings
    return [movie_id for movie_id, _ in final_recommendations[:top_n]]


In [114]:
# Call the function for a specific user (e.g., user 1)
home_page_recommendations_with_diversity(
    user_id=2, 
    ratings=ratings, 
    user_movie_matrix=user_movie_matrix, 
    item_item_recommendations=item_tem_recommendations, 
    user_user_recommendations=user_user_recommendations, 
    svd_prediction_matrix=svd_prediction_matrix
)


[356,
 69122,
 64614,
 127098,
 1682,
 63131,
 85342,
 139642,
 95843,
 105504,
 116797,
 140523,
 58303,
 4993,
 134130,
 5328,
 89745,
 122904,
 187717,
 4995,
 3941,
 67734,
 80463,
 2959,
 48780,
 79592,
 60885,
 7153,
 107348,
 6983,
 2571]

####  User Watches a New Movie

In [115]:
import random

def recommendations_after_new_movie(
    user_id, 
    new_movie_id, 
    ratings, 
    item_item_recommendations, 
    svd_prediction_matrix, 
    user_movie_matrix, 
    user_id_to_index, 
    index_to_movie_id, 
    weights=(0.7, 0.2), 
    top_n=10,
    rating_threshold=1
):
    """
    Generate recommendations after a user watches a new movie.
    
    :param user_id: ID of the user
    :param new_movie_id: ID of the newly watched movie
    :param ratings: DataFrame with columns ['userId', 'movieId', 'rating']
    :param item_item_recommendations: Precomputed item-item recommendations
    :param svd_prediction_matrix: SVD-based prediction matrix
    :param user_movie_matrix: User-item matrix
    :param user_id_to_index: Mapping from user ID to index in the SVD matrix
    :param index_to_movie_id: Mapping from matrix indices to movie IDs
    :param weights: Weights for blending the recommendation methods (item-item, SVD)
    :param top_n: Number of movies to recommend
    :param rating_threshold: Minimum predicted rating threshold for SVD-based recommendations
    :return: List of recommended movie IDs
    """
    # Get the user's ratings history
    user_rated_movies = ratings[ratings['userId'] == user_id]['movieId'].tolist()
    
    # ----- 1. Item-Item Collaborative Filtering -----
    similar_movies = []
    if new_movie_id in item_item_recommendations:
        similar_movies = item_item_recommendations[new_movie_id]
    
    # Remove movies the user has already rated
    similar_movies = [m for m in similar_movies if m not in user_rated_movies]
    
    # ----- 2. Predicted Ratings (SVD) -----
    svd_recommendations = []
    user_index = user_id_to_index[user_id]
    if user_index in user_movie_matrix.index:
        svd_predictions = svd_prediction_matrix[user_index]
        svd_movie_ids = [index_to_movie_id[idx] for idx in user_movie_matrix.columns]
        svd_recommendations = [
            movie_id for movie_id, pred in zip(svd_movie_ids, svd_predictions) 
            if pred > rating_threshold and movie_id not in user_rated_movies
        ]
    
    # ----- 3. Global Top Rated Movies -----
    global_top_movies = (
        ratings.groupby('movieId')['rating']
        .mean()
        .sort_values(ascending=False)
        .index.tolist()
    )
    
    # Remove movies the user has already rated
    global_top_movies = [m for m in global_top_movies if m not in user_rated_movies]
    
    # ----- Blending Recommendations -----
    item_item_count = int(weights[0] * top_n)
    svd_count = int(weights[1] * top_n)
    global_count = top_n - item_item_count - svd_count
    
    # Select the recommendations
    recommendations = (
        similar_movies[:item_item_count] +
        svd_recommendations[:svd_count] +
        global_top_movies[:global_count]
    )
    
    print(f"{len(similar_movies[:item_item_count])} / {item_item_count}")
    print(f"{len(svd_recommendations[:svd_count])} / {svd_count}")
    print(f"{len(global_top_movies[:global_count])} / {global_count}")
    
    # Deduplicate and introduce diversity
    recommendations = list(dict.fromkeys(recommendations))
    
    # Ensure the first 5 movies are strictly similar movies
    top_similar = similar_movies[:5]
    remaining_recommendations = [movie for movie in recommendations if movie not in top_similar]
    random.shuffle(remaining_recommendations)
    
    # Combine and limit to top_n
    final_recommendations = top_similar + remaining_recommendations[:top_n - len(top_similar)]
    
    # If not enough recommendations, fill with additional top-rated movies
    additional_movies = [m for m in global_top_movies if m not in final_recommendations]
    while len(final_recommendations) < top_n and additional_movies:
        final_recommendations.append(additional_movies.pop(0))
    
    return final_recommendations[:top_n]


In [116]:
recommendations = recommendations_after_new_movie(
    user_id=10,
    new_movie_id=20,
    ratings=ratings,
    item_item_recommendations=item_tem_recommendations,
    svd_prediction_matrix=svd_prediction_matrix,
    user_movie_matrix=user_movie_matrix,
    user_id_to_index=user_id_to_index,
    index_to_movie_id=index_to_movie_id,
    weights=(0.7, 0.2),
    top_n=20
)

recommendations

5 / 14
4 / 4
2 / 2


[20,
 27478,
 32213,
 26870,
 95583,
 187717,
 39,
 1,
 6983,
 318,
 293,
 5328,
 95843,
 3941,
 3940,
 3939,
 7815,
 162414,
 162344,
 158882]